In [1]:
# Re-import necessary libraries as execution state was reset
import numpy as np
import matplotlib.pyplot as plt
import random

# Define the map size
map_size = (100, 100)

# Define start and goal points
start = (10, 10)
goal = (90, 90)
goal_radius = 5  # How close we need to be to the goal to stop

# Define obstacles as circles (center_x, center_y, radius)
obstacles = [
    (40, 40, 10),
    (60, 60, 15),
    (30, 70, 10),
    (70, 20, 12)
]

# RRT parameters
step_size = 5
max_iterations = 500

# Helper function to check collision with obstacles
def is_collision_free(point):
    x, y = point
    for (ox, oy, r) in obstacles:
        if np.sqrt((x - ox) ** 2 + (y - oy) ** 2) < r:
            return False  # Collision detected
    return True

# RRT algorithm
class RRT:
    def __init__(self, start, goal, step_size, max_iterations):
        self.start = start
        self.goal = goal
        self.step_size = step_size
        self.max_iterations = max_iterations
        self.nodes = [start]  # List of explored nodes
        self.parent = {start: None}  # Parent dictionary for backtracking

    def get_random_point(self):
        """Generate a random point in the map."""
        return (random.randint(0, map_size[0]), random.randint(0, map_size[1]))

    def get_nearest_node(self, point):
        """Find the nearest node in the tree."""
        return min(self.nodes, key=lambda n: np.linalg.norm(np.array(n) - np.array(point)))

    def steer(self, from_node, to_node):
        """Move from from_node toward to_node by step_size."""
        vec = np.array(to_node) - np.array(from_node)
        length = np.linalg.norm(vec)
        if length == 0:
            return from_node
        direction = vec / length
        new_node = np.array(from_node) + self.step_size * direction
        return tuple(new_node.astype(int))

    def run(self):
        """Run the RRT algorithm to find a path."""
        for _ in range(self.max_iterations):
            rand_point = self.get_random_point()
            nearest_node = self.get_nearest_node(rand_point)
            new_node = self.steer(nearest_node, rand_point)

            if is_collision_free(new_node):
                self.nodes.append(new_node)
                self.parent[new_node] = nearest_node

                # Check if we reached the goal region
                if np.linalg.norm(np.array(new_node) - np.array(self.goal)) < goal_radius:
                    self.parent[self.goal] = new_node
                    return self.get_path()

        return None  # No path found

    def get_path(self):
        """Backtrack from goal to start to extract the path."""
        path = []
        node = self.goal
        while node is not None:
            path.append(node)
            node = self.parent[node]
        return path[::-1]  # Reverse to get start-to-goal path

# Run RRT
rrt = RRT(start, goal, step_size, max_iterations)
path = rrt.run()

# Visualization
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_xlim(0, map_size[0])
ax.set_ylim(0, map_size[1])

# Plot obstacles
for (ox, oy, r) in obstacles:
    circle = plt.Circle((ox, oy), r, color='red', alpha=0.5)
    ax.add_patch(circle)

# Plot RRT tree
for node, parent in rrt.parent.items():
    if parent:
        plt.plot([node[0], parent[0]], [node[1], parent[1]], 'g-', alpha=0.5)

# Plot path
if path:
    path_x, path_y = zip(*path)
    plt.plot(path_x, path_y, 'b-', linewidth=2, label="Path")

# Plot start and goal
plt.scatter(*start, color="blue", s=100, label="Start")
plt.scatter(*goal, color="green", s=100, label="Goal", marker="X")

plt.legend()
plt.title("RRT Path Planning in 2D")
plt.grid()
plt.show()


KeyboardInterrupt: 